<a href="https://colab.research.google.com/github/gyuholee94/NLP/blob/master/09_Prac_Document_Summarization(%EB%82%B4%EA%B0%80_%ED%95%9C%EA%B2%83).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9장. 문서 요약 (Text Summarization)

## Mecab 설치 (필요시)

In [1]:
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package openjdk-7-jdk is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'openjdk-7-jdk' has no installation candidate
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     |████████████████████████████████| 19.4MB 45.8MB/s 
     |████████████████████████████████| 92kB 6.4MB/s 
     |████████████████████████████████| 3.8MB 46.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing ins

# 9-1 Luhn Summarizer

http://courses.ischool.berkeley.edu/i256/f06/papers/luhn58.pdf

Hans Peter Luhn

https://en.wikipedia.org/wiki/Hans_Peter_Luhn

 Hans Peter Luhn은 공학과 정보과학에서의 개척 작업으로 "정보 검색의 아버지"로 알려져 있다. 그는 표제어가 문맥에 포함된 채 배열된 색인(KWIC : keyword-in-context) 개발, 정보 선택 제공(SDI), 완전 텍스트 프로세싱, 자동 발췌(요약), 단어 시소로스의 최초 현대식 사용으로 신뢰를 얻었다. 오늘날 파생된 지식 대부분에는 KWIC 색인이 있으며 과학의 모든 분야에 SDI시스템이 있다. 


 Luhn은 1896년 7월 1일 독일 바르멘에서 태어났다. 아버지가 그 당시 유명한 인쇄업자였으므로, 스위스에서 인쇄업을 배웠다. 어렸을 때무터 그는 창조성이 뛰어난 재능을 보였으며, 기술적 문제, 물리학, 통계학에 관심을 보였다. 그러나 1차 세계대전으로 독일군 통신장교로 복역(1915년1917년)하면서 프랑스 터키 루마니아 불가리아 등지로 옮겨 다녀야만 했다. 1차 대전 이후 그는 스위스의 Gallen 교회 Schweizwrische handels Hochschule로 돌아와 기술, 물리학, 회계분야의 수업을 들었다. 그 후, Luhn은 그리스에서 못다한 공부에 전념했으며, 더블부기기계(Duble-Entry Bookeeping Machine : 카드 대장에 대변기입과 차변 기입을 기록할 수 있는 것)를 발명하였다. 그는 또 Hollerith tabulating/recording machine에 정통했고, 천공카드에서 문자 숫자를 나타내는 장치의 사용을 증가시키게 했다. 1920년부터 1930년까지는 10개의 특허권을 획득하여 그의 탁월한 능력을 보여주었다. 그것들중에 루노메터(Lunometer : 직물의 실길이를 계산하는데 쓰이는 장치)는 지금도 사용되고 있다.


 1920년까지 그는 직물 공장에서 일하기 시작했다. 그는 직물 공장의 사업확장을 위해 뉴잉글랜드에서 1924년 미국으로 가게 되었다. 그러나 회사가 곧 파산하였고 Luhn은 직장없이 뉴욕에 남게 되었다. 그는 은행에서 일을 하였고 곧 뉴욕 월스트리트에 소재한 국제어음은행(International Acceptance Bank)에서 재정담당관으로 승진하였다. 


 1933년 Luhn은 자사인 공학회사 H.P. Luhn & Association을 설립하였고 8년간 자문기술자로 일했다. 1941년 Luhn은 IBM에서 수석 연구기술자로 참여하였고 이후에 정보검색연구 관리자로 일했다. Luhn이 IBM에서 새로운 아이디어를 지속적으로 내놓고 문제를 다르게 접근하여 주목을 받는 동안, 다른 기술자들에게 고차원적인 창조를 하도록 자극하면서 그들의 촉매제 역할을 하여 신뢰를 얻었다

In [2]:
import requests 
from bs4 import BeautifulSoup

def get_news_by_url(url):
  res = requests.get(url)
  bs = BeautifulSoup(res.content, 'html.parser')

  title = bs.select('h3#articleTitle')[0].text #제목
  content = bs.select('#articleBodyContents')[0].get_text().replace('\n', " ") #본문
  content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
  return  content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc

'과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자] 국가 차원의 빅데이터 활용 시대가 열린다. 새로운 산업 창출과 기존 산업의 변화에 이르는 ‘혁신성장’을 위한 센터가 문을 연다. 10개 분야에 걸쳐 ‘데이터 경제’의 발전을 위한 정부의 청사진을 현실로 구현하는데 앞장선다는 계획이다.22일 과학기술정보통신부는 서울 중구 대한상공회의소에서 데이터 생태계 조성과 혁신 성장의 기반 마련을 위한 ‘빅데이터 플랫폼 및 센터’ 출범식 행사를 개최했다. 유영민 과기정통부 장관을 비롯해 노웅래 국회 과학기술정보방송통신위원회 위원장 등 300여명이 참가했다.◇10개 분야 100개 센터..3년간 1516억원 투입이미지: 픽사베이빅데이터는 데이터 활용을 통해 혁신성장을 이루자는 문재인 정부의 경제 성장 핵심 요소중 하나다. 문재인 대통령이 직접 올 들어 데이터 활용과 이에 따른 정보보호(보안)에 대한 중요성을 강조하기도 했다.이런 맥락 속에서 빅데이터센터는 공공과 민간이 협업해 활용도 높은 양질의 데이터를 생산·구축하고, 플랫폼은 이를 수집·분석·유통하는 역할을 담당한다. 과기정통부는 분야별 플랫폼 10개소와 이와 연계된 기관별 센터 100개소를 구축하는데 3년간 총 1516억원을 투입할 계획이며, 올해 우선 640억원 규모의 사업을 추진하고 있다.대상 분야는 △금융(BC카드) △환경(한국수자원공사) △문화(한국문화정보원) △교통(한국교통연구원) △헬스케어(국립암센터) △유통·소비(매일방송) △통신(KT) △중소기업(더존비즈온) △지역경제(경기도청) △산림(한국임업진흥원) 등으로 현재 1차 공모를 통해 72개 빅데이터 센터를 선정했고, 다음달 8일까지 2차 공모를 통해 28개를 추가 선정해 총 100개를 지원, 운영할 계획이다. 이를 통해 데이터 생태계를 혁신하고 기업의 경쟁력을 제고하는 역할을 수행한다.주요 활용 전략·사례를 보면 빅데이터 활용을 

In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 1) 토큰화

In [4]:
   # word_list = []
  # for sentence in get_sentences(txt):
  #   for token in mecab.pos(sentence):
  #       if token[1][0] != 'S':
  #         word_list.append(token)
  #nodes = [token for token in mecab.pos(txt)]# if token[1] in ['NNG','NNP']]
  

In [5]:
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab

def get_sentences(txt):
  sentence = sent_tokenize(txt)
  return sentence   

def get_words(txt):
  mecab = Mecab()
  tokens = [token for sentence in get_sentences(txt) for token in mecab.pos(sentence) if token[1][0] != 'S']# or '']
  return tokens


print(len(get_sentences(doc)))
get_words(get_sentences(doc)[0])

12


[('과기', 'NNG'),
 ('정통부', 'NNG'),
 ('일', 'NNBC'),
 ('유영민', 'NNP'),
 ('장관', 'NNG'),
 ('등', 'NNB'),
 ('참석', 'NNG'),
 ('해', 'XSV+EC'),
 ('기념행사', 'NNG'),
 ('년', 'NNBC'),
 ('까지', 'JX'),
 ('억', 'NR'),
 ('원', 'NNBC'),
 ('투입', 'NNG'),
 ('여종', 'NNG'),
 ('데이터', 'NNG'),
 ('구축', 'NNG'),
 ('민간', 'NNG'),
 ('클라우드', 'NNP'),
 ('통한', 'VV+ETM'),
 ('외부', 'NNG'),
 ('연계', 'NNG'),
 ('체계', 'NNG'),
 ('도', 'JX'),
 ('개방', 'NNG'),
 ('성', 'XSN'),
 ('강화', 'NNG'),
 ('이', 'JKS'),
 ('데일리', 'NNP'),
 ('이재운', 'NNP'),
 ('기자', 'NNG'),
 ('국가', 'NNG'),
 ('차원', 'NNG'),
 ('의', 'JKG'),
 ('빅', 'NNG'),
 ('데이터', 'NNG'),
 ('활용', 'NNG'),
 ('시대', 'NNG'),
 ('가', 'JKS'),
 ('열린다', 'VV+EF')]

In [6]:
total_tokens= []
for i in range(len(get_words(doc))):
  total_tokens.append(get_words(doc)[i][0])
total_tokens

['과기',
 '정통부',
 '일',
 '유영민',
 '장관',
 '등',
 '참석',
 '해',
 '기념행사',
 '년',
 '까지',
 '억',
 '원',
 '투입',
 '여종',
 '데이터',
 '구축',
 '민간',
 '클라우드',
 '통한',
 '외부',
 '연계',
 '체계',
 '도',
 '개방',
 '성',
 '강화',
 '이',
 '데일리',
 '이재운',
 '기자',
 '국가',
 '차원',
 '의',
 '빅',
 '데이터',
 '활용',
 '시대',
 '가',
 '열린다',
 '새로운',
 '산업',
 '창출',
 '과',
 '기존',
 '산업',
 '의',
 '변화',
 '에',
 '이르',
 '는',
 '혁신',
 '성',
 '장',
 '을',
 '위한',
 '센터',
 '가',
 '문',
 '을',
 '연다',
 '개',
 '분야',
 '에',
 '걸쳐',
 '데이터',
 '경제',
 '의',
 '발전',
 '을',
 '위한',
 '정부',
 '의',
 '청사진',
 '을',
 '현실',
 '로',
 '구현',
 '하',
 '는',
 '데',
 '앞장선다는',
 '계획',
 '이',
 '다',
 '일',
 '과학',
 '기술',
 '정보',
 '통신부',
 '는',
 '서울',
 '중구',
 '대한',
 '상공',
 '회의소',
 '에서',
 '데이터',
 '생태',
 '계',
 '조성',
 '과',
 '혁신',
 '성장',
 '의',
 '기반',
 '마련',
 '을',
 '위한',
 '빅',
 '데이터',
 '플랫',
 '폼',
 '및',
 '센터',
 '출범식',
 '행사',
 '를',
 '개최',
 '했',
 '다',
 '유영민',
 '과기',
 '정통부',
 '장관',
 '을',
 '비롯',
 '해',
 '노웅래',
 '국회',
 '과학',
 '기술',
 '정보',
 '방송',
 '통신',
 '위원회',
 '위원장',
 '등',
 '여',
 '명',
 '이',
 '참가',
 '했',
 '다',
 '개',
 '분야',
 '개',


In [7]:
lines_tokens = []

for i in range(len(get_sentences(doc))):
  line_tokens = []
  for j in range(len(get_words(get_sentences(doc)[i]))):
    line_tokens.append(get_words(get_sentences(doc)[i])[j][0])
  lines_tokens.append(line_tokens)
len(lines_tokens)

12

In [8]:
get_words(get_sentences(doc)[1])[0][0]

'새로운'

In [9]:
get_words(get_sentences(doc)[0])

[('과기', 'NNG'),
 ('정통부', 'NNG'),
 ('일', 'NNBC'),
 ('유영민', 'NNP'),
 ('장관', 'NNG'),
 ('등', 'NNB'),
 ('참석', 'NNG'),
 ('해', 'XSV+EC'),
 ('기념행사', 'NNG'),
 ('년', 'NNBC'),
 ('까지', 'JX'),
 ('억', 'NR'),
 ('원', 'NNBC'),
 ('투입', 'NNG'),
 ('여종', 'NNG'),
 ('데이터', 'NNG'),
 ('구축', 'NNG'),
 ('민간', 'NNG'),
 ('클라우드', 'NNP'),
 ('통한', 'VV+ETM'),
 ('외부', 'NNG'),
 ('연계', 'NNG'),
 ('체계', 'NNG'),
 ('도', 'JX'),
 ('개방', 'NNG'),
 ('성', 'XSN'),
 ('강화', 'NNG'),
 ('이', 'JKS'),
 ('데일리', 'NNP'),
 ('이재운', 'NNP'),
 ('기자', 'NNG'),
 ('국가', 'NNG'),
 ('차원', 'NNG'),
 ('의', 'JKG'),
 ('빅', 'NNG'),
 ('데이터', 'NNG'),
 ('활용', 'NNG'),
 ('시대', 'NNG'),
 ('가', 'JKS'),
 ('열린다', 'VV+EF')]

### 2) 중요 단어 결정

In [10]:
# 단어(토큰)의 가중치 계산 및 범위에 포함되는 토큰 식별 
def get_keywords(word_list , min_ratio=0.03, max_ratio=0.5) :
  keywords = []
  for word in word_list:  
    if word_list.count(word)/len(word_list) < max_ratio and word_list.count(word)/len(word_list) > min_ratio:
        keywords.append(word)
  return keywords

get_keywords(['바나나','바나나','사과','바나나','포도'])

['사과', '포도']

In [11]:
a = ['바나나','사과','바나나','바나나','사과','바나나','포도']
a.count('포도')/len(a)

0.14285714285714285

In [13]:
keywords = get_keywords(tokens)

NameError: ignored

### 3) 문장 중요도 계산

In [14]:
# 문장의 가중치 계산
def get_sentence_weight (sentence , keywords):
  for i in range(len(sentence)):
    for j in range(len(sentence[i])):
      sentence[i][j]
  return 

In [15]:
get_sentence_weight(get_sentences(doc), keywords)

NameError: ignored

In [16]:
len(keywords)


NameError: ignored

In [ ]:
len(lines_tokens[1])

In [ ]:
lines_freq = []


for i in range(len(lines_tokens)):
  line_freq = []
  for idx, j in enumerate(lines_tokens[i]):
    if j not in keywords:
      line_freq.append(idx)
    else:
      continue
  lines_freq.append(line_freq)
lines_freq

In [17]:
result = []
for i in range(len(lines_freq)):
  result.append(len(lines_freq[i]) * len(lines_freq[i])/(lines_freq[i][-1] - lines_freq[i][0]))
result

NameError: ignored

### 4) 문서 요약

In [ ]:
# 문서 요약
def summarize(content ,max_no_of_sentences = 10):
    
    pass

In [ ]:
li = summarize (doc ,  3)
for s in li :
    print(s)



---



# 9-2 Textrank

![대체 텍스트](https://www.researchgate.net/profile/Khushboo_Thakkar3/publication/232645575/figure/fig1/AS:575720050573312@1514273764062/Sample-graph-build-for-sentence-extraction.png)

## 2.1 TextRank 직접 구현하기 (Matrix 활용)

### 1) 자카드 유사도

In [ ]:
Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [ ]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
  
  Jac = len(set(set(sentence1) & set(sentence2))) / len(set(sentence1 + sentence2))
  return Jac

# sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

In [ ]:
sentence_similarity('나는 치킨을 좋아해', '나는 치킨을 싫어해')

In [ ]:
from nltk.tokenize import sent_tokenize
from konlpy.tag import Mecab

def get_sentences(txt):
  sentence = sent_tokenize(txt)
  
  return sentence

print(get_sentences(Text))


In [ ]:
for i in text[0]:
  print(i.replace('.', ''))

In [ ]:
text = [i.split() for i in get_sentences(Text)]
for i in range(len(text)):
  text[i][-1] = text[i][-1].replace('.', '')
text

In [ ]:
len(text)

In [ ]:
set(text[0]) &set(text[3])

In [ ]:
sentence_similarity(text[0], text[3])

In [ ]:
mat = np.zeros((len(text), len(text)), dtype=float)
text_len = len(text)
for i in range(text_len):
  for j in range(text_len):
    mat[i , j] = sentence_similarity(text[i], text[j])
    mat[j , i] = sentence_similarity(text[j], text[i])
    mat[i , i] = 0

In [ ]:
mat

In [ ]:
for i in range(len(mat)):
  

In [ ]:
score = mat.sum(axis=1)
score[0]

In [ ]:
score.T

In [ ]:
next = np.zeros((len(text), len(text)), dtype=float)
for i in range(text_len):
  for j in range(text_len):
    next[i , j] = mat[i, j]/score[i]
    # next[j , i] = mat[j, i]/score[i]
    next[i , i] = 0
print(score)
print(next)

In [ ]:
token = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

vocab2idx = {vocab[i]:i for i in range(len(vocab))}#vocab을 인덱스로 변환



idx2vocab = {i:vocab[i] for i in range(len(vocab))}  #인덱스를 vocab으로 변환
idx2vocab

import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((len(vocab), len(vocab)), dtype=float)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones(len(vocab), dtype=float)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
# covered_coocurrences = []
for i in range(len(token)-window_size +1):
  
  weighted_edge[vocab2idx[token[i:i+window_size][0]], vocab2idx[token[i:i+window_size][1]]] +=1
  weighted_edge[vocab2idx[token[i:i+window_size][1]], vocab2idx[token[i:i+window_size][0]]] +=1

k = np.zeros((len(vocab), len(vocab)), dtype=float)

for a in range(len(vocab2idx)):
  for b in range(len(vocab2idx)):
    k[a, b] = weighted_edge[a][b]/weighted_edge.sum(axis=1)[a]
weighted_edge = k  

weighted_edge

### 2) 그래프 생성

In [ ]:
toc_sent = [toc for toc in sent_tokenize(Text)]

In [ ]:
toc_sent[0].split()

In [ ]:
len(toc_sent)

In [25]:
import numpy as np
toc_len = len(toc_sent)
toc_sent = [toc for toc in sent_tokenize(Text)]
weighted_edge = np.zeros((toc_len, toc_len), dtype=float)
for i in range(toc_len):
  for j in range(toc_len):
    print(sentence_similarity(toc_sent[i], toc_sent[j]))
  
Text = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

weighted_edge

NameError: ignored

In [ ]:
스코어 어떻게 쓰라는거야..

### 3) 문장 중요도 계산

In [ ]:
def scoring(A, P, eps=0.0001, d=0.85, max_iter = 50):
  pass

### 4) 문서 요약

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize

def summarize(text, n=10):
  pass

In [ ]:
summary = summarize(Text, 3)

for sent in summary :
  print(sent)



---



## 2.2 TextRank 직접 구현하기 (Graph 활용)

In [ ]:
Text = "딸기 바나나 사과 딸기 파인애플. 바나나 사과 딸기. 복숭아 파인애플. 파인애플 사과 딸기 바나나."

### 2) 토큰화

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

In [18]:
def sentok(text):
  token = sent_tokenize(text)

  return token

In [19]:
sent_tokenize(Text)

NameError: ignored

In [ ]:
def sentences(text):
    return sent_tokenize(text)

### 3) 자카드 유사도

In [20]:
from konlpy.tag import Mecab

# 문장간 유사도 측정 (자카드 유사도 사용)
def sentence_similarity(sentence1, sentence2):
  pass

sentence_similarity('나는 치킨을 좋아해','나는 치킨을 싫어해')

In [21]:
def connect(nodes):
    pass

### 3) 그래프 생성

In [22]:
import networkx as nx

def rank(nodes,edges):
    pass

### 4) 문서 요약

In [23]:
def summarize(text,num_summaries=3):
    pass


In [24]:
summary=summarize(Text, 3)

for sent in summary :
  print(sent)

NameError: ignored

## 2.3 gensim을 활용한 요약

In [ ]:

print(docs[1])

In [ ]:
from gensim.summarization.summarizer import summarize
summarize(docs[1]).split('\n')



---

